

Your groups have been assigned by Nationwide, and now it is time to begin working together to build towards your Capstone presentations! Your first group project will be to gather the additional county auditor data, clean it, merge it, and begin to understand it for those counties that are considered part of the Columbus Ohio MSA: Franklin, Fairfield, Licking, Delaware, Hocking, Madison, Morrow, Perry, Pickaway, and Union. We have already worked with Franklin, Fairfield, and Licking County auditor datasets during Unit 2, and will continue working with these datasets, but your job as a group is to combine all pf the county datasets together.

A copy of auditor data for all of these counties is attached to this entry on Blackboard. Note that Franiklin, Fairfield, and Licking county data is not included since that data was already made available in Unit 2 Exercises. Additionally, Union County data is unavailable at this time - Bonus points to any team who can figure out how to get it!

Group Project 1 presentations will be at the beginning of class on June 20, and are to be 10 minutes or less in length. Presentations for this first group project can be done by one ore more members of the group. You will be graded on the following for this first group presentation:

    Presentation of basic statistics and charts showing how your team chose to sample (or not sample) data files, what format you stored it in, and any interesting facts you found while reviewing basic statistics about the data.
    What your team would like to study next about the data.
    Presentation succinctness: less than 10min in duration, highlight main points, highlight any key questions or concerns about the data that your team had as you performed the work.

Remember that this data can and should be used by your group during your Capstone work, so take your time building it well. This is your team's chance to perform its first practice run at presenting to an audience - enjoy it! The pressure to have robust data and impressive data visualization and modeling techniques can wait until your Capstone.


In [79]:
# Sampling functions
import linecache
import random
import samplingu2 as sampling 
import pandas as pd

###
### Delaware County
###

# pandas settings
pd.set_option('display.max_column', 200)

# load full dataset for initial analysis. Delaware uses "|" as a delimiter
delaware = pd.read_csv('../data/county_auditor/OH-Delaware/governmaxextract.txt', delimiter="|", low_memory=False)

# sample the data (default is 10% of all rows)
#sampling.sample_file('../data/county_auditor/OH-Delaware/governmaxextract.txt', 
#            '../data/county_auditor/OH-Delaware/delaware_county_subset.txt')



# Filter Data
# Delaware as column called Parcel_Type, but all the values in the column are 0.
# Instead the column called mClassificationId contains Land Use Codes, which we use as a filter.
# I'm assuming the data of interest is Residental Property. According to http://codes.ohio.gov/oac/5703-25-10
# codes between 500 and 599 are Taxable residential real property
delaware_subset = delaware[delaware.mClassificationId >= 500]
delaware_subset = delaware_subset[delaware_subset.mClassificationId <= 599]


display(delaware_subset.head())


# Filter out sales above 1.5 million dollars and sales of zero dollars
delaware_subset = delaware_subset[delaware_subset.SaleAmount < 1500000]
delaware_subset = delaware_subset[delaware_subset.SaleAmount > 0]

# Delaware separates out the address across six columns. Combine them into one Street Address
street_address = delaware_subset.mlocStrDir.fillna('') + " "
street_address += delaware_subset.mlocStrNo.fillna('') + " "
street_address += delaware_subset.mlocStrNo2.fillna('') + " "
street_address += delaware_subset.mlocStrName.fillna('') + " "
street_address += delaware_subset.mlocStrSuffix.fillna('') + " "
street_address += delaware_subset.mlocStrSuffixDir.fillna('') + " "
street_address += delaware_subset.mlocCity.fillna('') + " "
street_address += delaware_subset.mlocState.fillna('') + " "
street_address += delaware_subset.mlocZipCode.fillna('')

# Add the street_address to the dataframe at index 2
delaware_subset.insert(2, "PropertyAddress", street_address)

#restrict the columns of interest
delaware_columns = [ 'mpropertyNumber', 'OwnName', 'PropertyAddress',
                     'mClassificationId','macres', 'mlegalDescription', 
                    'NeighborhoodCode', 'SaleDate', 'SaleAmount']

# Apply the column masks to the dataframe
delaware_subset = delaware_subset[delaware_columns].copy()

delaware_subset.rename(
    {'mpropertyNumber' : 'ParcelNumber',     
     'OwnName' : 'Owner',
     'mlocDescription' : 'Description',
     'mClassificationId' : 'LandUse',
     'macres' : 'Acres',     
     'mlegalDescription' : 'LegalDescription',
     'SaleAmount':'SalePrice'
    },
    axis=1,
    inplace=True
)

# Add Columns for data merge
mailing_address = delaware_subset.PropertyAddress
delaware_subset.insert(3, "MailingAddress", mailing_address)

delaware_subset.insert(10, "YearBuilt", " ")
delaware_subset.insert(11, "NumberOfStories", " ")
delaware_subset.insert(12, "FinishedArea", " ")
delaware_subset.insert(13, "NumberOfRooms", " ")
delaware_subset.insert(14, "NumberOfBedrooms", " ")
delaware_subset.insert(15, "NumberOfFullBaths", " ")
delaware_subset.insert(16, "NumberOfHalfBaths", " ")

delaware_subset['County'] = 'Delaware'

#for column in delaware_subset.columns:
#    print(column)
display(delaware_subset.head())
display(delaware_subset.YearBuilt.unique())

,mpropertyNumber,mmapNumber,mcamaId,CardCount,NeighborhoodCode,mlegalDescription,mlocStrDir,mlocStrNo,mlocStrNo2,mlocStrName,mlocStrSuffix,mlocStrSuffixDir,msecondaryAddress,mlocCity,mlocState,mlocZipCode,mlocDescription,mClassificationId,macres,mfrontfootage,mdeedVolume,mdeedPage,Par_Deleted,DeededOwner,mlot,msublot,OwnName,OwnFirstName,OwnMiddleInitial,OwnLastName,OwnNameSuffix,OwnBusiness,OwnStreetName,OwnStreetNumber,OwnStreetDirection,OwnStreetSuffix,OwnSufDir,OwnSecondaryAddress,OwnCity,OwnState,OwnZipcode,OwnCountry,OwnAttentionline,TaxpName,TaxpFirstName,TaxpMiddleInitial,TaxpLastName,TaxpNameSuffix,TaxpBusiness,TaxpStreetName,TaxpStreetNumber,TaxpStreetDirection,TaxpStreetSuffix,TaxpStreetSuffixDir,TaxpSecondaryAddress,TaxpCity,TaxpState,TaxpZipcode,TaxpCountry,TaxpAttentionline,Spec_Flag,mtaxsetCode,mstateSchoolcode,mvaluedBy,MKT_Land_Value,MKT_Impr_Value,CAUV_Value,MKT_Total_Value,MKT_Tot_Total,SaleDate,ValidSale,SaleAmount,NumberOfPropertiesInSale,InstumentType,InstumentDate,Conveyance,DeedNumber,Hmstd_Flag,Foreclosed,NewConstruction,Reduction25,BORflag,DividedProperty,LandVal,ImproveVal,SpecAssessment,AnnualTax,TaxesPaid,Delinquent,mroutingNumber,Zoning,Parcel_Type,ForeclosureStep,TresCode,BC,activeYear,LenderId,TaxLienStatus,OwnOtherInfo,mFORCmessageid,msubdivision
3,100-100-01-001-000,NaN,NaN,1.0,38002.0,LANDS SURVEY 675 ^ ...,NaN,7511,NaN,DAVIS-KIRK,RD,NaN,NaN,PROSPECT,OH,43342,NaN,511.0,1.00,0.0,0049,1746,N,BURNS JASON E HORNER JACQUELINE S,,,BURNS JASON E,JASON,E,BURNS,NaN,0.0,DAVIS-KIRK,7511,NaN,RD,NaN,NaN,PROSPECT,OH,43342,USA,HORNER JACQUELINE S,CORELOGIC,CORELOGIC,,NaN,NaN,0.0,HACKBERRY,3001,NaN,NaN,NaN,NaN,IRVING,TX,75063,USA,NaN,N,39.0,5101.0,RVL,23800,146200,0,170000,170000,09-12-2000,Y,127000,1,16,2000-09-12 12:00:00,3898,NaN,N,N,N,Y,N,N,8330,51170,0,2367.34,2367.34,0.0,NaN,,0,NaN,TU1213,NaN,1990,1521,NaN,,0,0
4,100-100-01-002-000,NaN,NaN,1.0,38002.0,LANDS SURVEY 675 ^ ...,NaN,7491,NaN,DAVIS-KIRK,RD,NaN,NaN,PROSPECT,OH,43342,NaN,511.0,1.00,0.0,NaN,NaN,N,REEBEL RICK R TRUSTEE,,,REEBEL RICK R TRUSTEE,REEBEL,R,R,NaN,0.0,DAVIS-KIRK,7491,NaN,RD,NaN,NaN,PROSPECT,OH,43342,USA,NaN,REEBEL RICK R TRUSTEE,REEBEL,R,R,NaN,0.0,DAVIS-KIRK,7491,NaN,RD,NaN,NaN,PROSPECT,OH,43342,USA,NaN,N,39.0,5101.0,RVL,23800,115100,0,138900,138900,05-15-2018,N,0,2,18,2018-05-15 09:58:00,NaN,912,Y,N,N,Y,N,N,8330,40290,0,1586.94,1586.94,0.0,NaN,,0,NaN,NaN,NaN,1990,0,NaN,NaN,0,0
5,100-100-01-003-000,NaN,NaN,0.0,38002.0,LANDS SURVEY 675 ^ ...,NaN,NaN,NaN,DAVIS-KIRK,RD,NaN,NaN,PROSPECT,OH,43342,NaN,511.0,0.47,0.0,NaN,NaN,N,REEBEL RICK R TRUSTEE,,,REEBEL RICK R TRUSTEE,REEBEL,R,R,NaN,0.0,DAVIS-KIRK,NaN,NaN,RD,NaN,NaN,PROSPECT,OH,43342,USA,NaN,REEBEL RICK R,REEBEL,R,R,NaN,0.0,DAVIS KIRK,7491,NaN,RD,NaN,NaN,PROSPECT,OH,43342,USA,TRUSTEE,N,39.0,5101.0,RVL,8800,0,0,8800,8800,05-15-2018,N,0,2,18,2018-05-15 09:58:00,NaN,912,N,N,N,N,N,N,3080,0,0,126.04,126.04,0.0,NaN,,0,NaN,T 12/0,NaN,1990,0,NaN,NaN,0,0
6,100-100-01-004-000,NaN,NaN,1.0,38002.0,LANDS SURVEY 675 ^ ...,NaN,7409,NaN,DAVIS-KIRK,RD,NaN,NaN,PROSPECT,OH,43342,NaN,599.0,5.00,0.0,NaN,NaN,N,RIDER DAVID R,,,RIDER DAVID R,DAVID,R,RIDER,NaN,0.0,DAVIS-KIRK,7409,NaN,RD,NaN,NaN,PROSPECT,OH,43342,USA,NaN,RIDER DAVID R,DAVID,R,RIDER,NaN,0.0,DAVIS-KIRK,7409,NaN,RD,NaN,NaN,PROSPECT,OH,43342,USA,NaN,N,39.0,5101.0,RVL,57200,0,0,57200,57200,08-01-1991,N,0,0,0,1991-08-01 12:00:00,0,NaN,Y,N,N,Y,N,N,20020,0,0,529.92,529.92,0.0,NaN,,0,NaN,TX 8/1,NaN,1990,0,NaN,,0,0
7,100-100-01-005-000,NaN,NaN,1.0,38002.0,LANDS SURVEY 675 ^ ...,NaN,7321,NaN,DAVIS-KIRK,RD,NaN,NaN,PROSPECT,OH,43342,NaN,511.0,0.80,0.0,NaN,NaN,N,VANHOESEN KEVIN D & ROBIN,,,VANHOESEN KEVIN D,VANHOESEN,K,D,NaN,0.0,DAVIS-KIRK,7321,NaN,RD,NaN,NaN,PROSPECT,OH,43342,USA,& ROBIN,CORELOGIC,CORELOGIC,,NaN,NaN,0.0,HACKBERRY,3001,NaN,NaN,NaN,NaN,IRVING,TX,75063,USA,NaN,N,39.0,5101.0,RVL,23200,93800,0,117000,117000,02-02-2016,Y,128500,1,16,2016-02-02 14:33:00,341,NaN,N,N,N,Y,N,N,8120,32830,0,1635.56,1635.56,0.0,NaN,,0,NaN,TU1213,NaN,1990,1521,NaN,NaN,0,0


,ParcelNumber,Owner,PropertyAddress,MailingAddress,LandUse,Acres,LegalDescription,NeighborhoodCode,SaleDate,SalePrice,YearBuilt,NumberOfStories,FinishedArea,NumberOfRooms,NumberOfBedrooms,NumberOfFullBaths,NumberOfHalfBaths,County
3,100-100-01-001-000,BURNS JASON E,7511 DAVIS-KIRK RD PROSPECT OH 43342,7511 DAVIS-KIRK RD PROSPECT OH 43342,511.0,1.000,LANDS SURVEY 675 ^ ...,38002.0,09-12-2000,127000,,,,,,,,Delaware
7,100-100-01-005-000,VANHOESEN KEVIN D,7321 DAVIS-KIRK RD PROSPECT OH 43342,7321 DAVIS-KIRK RD PROSPECT OH 43342,511.0,0.800,LANDS SURVEY 675 ^ ...,38002.0,02-02-2016,128500,,,,,,,,Delaware
13,100-100-01-010-001,PORTUESI BEVERLY J,7745 STATE ROUTE 257 N PROSPECT OH 43342,7745 STATE ROUTE 257 N PROSPECT OH 43342,511.0,1.492,LANDS SURVEY 675 ^TRACT 1 ...,38002.0,10-27-2015,19500,,,,,,,,Delaware
14,100-100-01-010-002,VALENCIA CARLOS PEREZ,7741 STATE ROUTE 257 N PROSPECT OH 43342,7741 STATE ROUTE 257 N PROSPECT OH 43342,501.0,1.492,LANDS SURVEY 675 ^TRACT 2 ...,38002.0,02-17-2017,19000,,,,,,,,Delaware
15,100-100-01-011-000,WILLIAMS ROBERT H III,7715 STATE ROUTE 257 N PROSPECT OH 43342,7715 STATE ROUTE 257 N PROSPECT OH 43342,511.0,2.000,LANDS SURVEY 675 ^ ...,38002.0,11-06-2013,250000,,,,,,,,Delaware


array([' '], dtype=object)

In [ ]:
### Sample County Data

# using sampleCounty in samplingsu2.py
delaware_subset = sampling.sampleCounty(delaware_subset, 0.1)

display(delaware_subset.head())

In [76]:
###
### Hocking County
###

hocking = pd.read_csv('../data/county_auditor/OH-Hocking/Grid_salef_2019-03-20.csv', delimiter=",", low_memory=False)
hocking_subset = hocking

# Function to change the values in a field
def rename_values(row, column, original, replacement):
    field = row[column]
    return field.replace(original, replacement).strip()

display(hocking_subset.head())

### Filters and Drops

# filter out any property that has NaN Sqft, House Number (Number) is NaN (Assume no house on property)
hocking_subset = hocking_subset[pd.notna(hocking_subset.Sqft)]
hocking_subset = hocking_subset[pd.notna(hocking_subset.Number)]

# drop
hocking_subset.drop(['Corp / Village'], axis=1, inplace=True)
hocking_subset.drop(['MHRE'], axis=1, inplace=True)

### Address
# Convert Number and Street to same data type
hocking_subset['Number'] = hocking_subset.Number.astype(str)
hocking_subset['Street'] = hocking_subset.Street.astype(str)

# Combine Number and Street to create Address
address = hocking_subset.Number.fillna('') + " "
address += hocking_subset.Street.fillna('')

# Add the street_address to the dataframe at index 2
hocking_subset.insert(1, "Address", address)

# drop Number and Street
hocking_subset.drop(['Number'], axis=1, inplace=True)
hocking_subset.drop(['Street'], axis=1, inplace=True)

### Bathrooms

# Combine FullBaths and HalfBaths to Bathrooms
#bathrooms = (hocking_subset.FullBaths.fillna(0) + 0.5 * hocking_subset.HalfBaths.fillna(0))
#hocking_subset.insert(hocking_subset.columns.get_loc('HalfBaths') - 1, "Bathrooms", bathrooms)

# Remove FullBaths, HalfBaths
#hocking_subset.drop(['FullBaths'], axis=1, inplace=True)
#hocking_subset.drop(['HalfBaths'], axis=1, inplace=True)

hocking_subset.FullBaths = hocking_subset.FullBaths.fillna(0)
hocking_subset.HalfBaths = hocking_subset.HalfBaths.fillna(0)

### Garage

# Rename Type to GarageType
#hocking_subset.rename({'Type' : 'GarageType'}, axis=1, inplace=True)
#hocking_subset.loc[:,'GarageType'] = hocking_subset.GarageType.fillna('None')

# Rename D to Detached and A to Attached
#hocking_subset.loc[:,'GarageType'] = hocking_subset.apply(rename_values, args=('GarageType','D','Detached'), axis=1)
#hocking_subset.loc[:,'GarageType'] = hocking_subset.apply(rename_values, args=('GarageType','A','Attached'), axis=1)

# GarageSqft: If NaN, then zero.
#hocking_subset.loc[:, 'GarageSqft'] = hocking_subset['GarageSqft'].fillna(0)

### PropClass aka Land Use Codes

# PropClass is Land Use Codes; only include Residental Properties
hocking_subset = hocking_subset[hocking_subset.PropClass >= 500]
hocking_subset = hocking_subset[hocking_subset.PropClass <= 599]

### Basement

# Basement: Convert NaN to 'No', B to 'Full' and H to 'Half'
#hocking_subset.loc[:,'Basement'] = hocking_subset.Basement.fillna('No')
#hocking_subset.loc[:,'Basement'] = hocking_subset.apply(rename_values, args=('Basement','B','Full'), axis=1)
#hocking_subset.loc[:,'Basement'] = hocking_subset.apply(rename_values, args=('Basement','H','Half'), axis=1)

# Basement Sqrt Footage: If NaN, replace with zero
#hocking_subset.loc[:,'BasementSqft'] = hocking_subset.BasementSqft.fillna('0')

# Basement Finished: If NaN, replace with No. Replace F with Finished
#hocking_subset.loc[:, 'Fin Base'] = hocking_subset['Fin Base'].fillna('No')
#hocking_subset.loc[:,'Fin Base'] = hocking_subset.apply(rename_values, args=('Fin Base','F','Finished'), axis=1)

# Bedrooms: If NaN, then 1
hocking_subset.loc[:, 'Bedrooms'] = hocking_subset['Bedrooms'].fillna(1)

### Air

# Air "Air Condition": If NaN, then No. If Y, then change to Yes
#hocking_subset.loc[:, 'Air'] = hocking_subset['Air'].fillna('No')
#hocking_subset.loc[:,'Air'] = hocking_subset.apply(rename_values, args=('Air','Y','Yes'), axis=1)


### Roofing

# Roof Type: For each RoofType as NaN, then RoofType is assigned to the highest type when Roof and RoofType are group.
# Use:
# roof_roofType = hocking_subset.groupby(['Roof', 'RoofType'])['Yr Built'].count()
# display(roof_roofType)
# to find the most common RoofType for each roof

#hocking_subset['Roof'] = hocking_subset.Roof.astype(object)
#hocking_subset.loc[:, 'Roof'] = hocking_subset['Roof'].fillna('O')
#hocking_subset.loc[:, 'Roof'] = hocking_subset.apply(rename_values, args=('Roof','`','O'), axis=1)

# Name NaN as unknown
#hocking_subset.loc[:, 'RoofType'] = hocking_subset['RoofType'].fillna('unknown')

# Replace unknow with common RoofType
#for index, row in hocking_subset.iterrows():
#    if isinstance(row.RoofType, str):
#        roof = row['Roof']
#        rtype = row['RoofType']
#        if rtype == 'unknown':
#            if roof == 'M':
#                new_value = row.RoofType.replace('unknown','MET')
#                hocking_subset.loc[index, 'RoofType'] = new_value
#            if roof == 'S':
#                new_value = row.RoofType.replace('unknown','ASP')
#                hocking_subset.loc[index, 'RoofType'] = new_value
#            if roof == 'O':
#                new_value = row.RoofType.replace('unknown','ROL')
#                hocking_subset.loc[index, 'RoofType'] = new_value
#            if roof == 'L':
#                new_value = row.RoofType.replace('unknown','SLA')
#                hocking_subset.loc[index, 'RoofType'] = new_value
                
### Rooms

# If Room is NaN, then set Median room
# Use:
#hocking_subset.loc[:, 'Rooms'] = hocking_subset['Rooms'].fillna('unknown')
#rooms_saleprice = hocking_subset.groupby(['Rooms', 'Yr Built'])['Sale Price'].median()
#display(rooms_saleprice)
# To determine median number of rooms

#hocking_subset.loc[:, 'Rooms'] = hocking_subset['Rooms'].fillna(5)

### Sale Date from Month and Year
# convert both to str
hocking_subset['Month'] = hocking_subset.Month.astype(str)
hocking_subset['Year'] = hocking_subset.Year.astype(str)

# concatenate Month and Year to create SaleDate
sale_date = hocking_subset.Month + "-01-20" + hocking_subset.Year
hocking_subset.insert(hocking_subset.columns.get_loc('Month') - 1, "SaleDate", sale_date)


### Rename Columns

# Remove spaces in names and clarify names
hocking_subset.rename(
    {
     'Parcel Number' : 'ParcelNumber',
     'Address' : 'PropertyAddress',
     'School District' : 'SchoolDistrict',
     'Sale #': 'SaleNumber',
     'Sale Price' : 'SalePrice',
     'Yr Built' : 'YearBuilt',
     'Fin Base' : 'FinishedBasement',
     'Heat Type' : 'HeatType',
     'Property Card' : 'PropertyCard',
     'PropClass' : 'LandUse',
     'NeighCode' : 'NeighborhoodCode',
     'Story' : 'NumberOfStories',
     'Sqft' : 'FinishedArea',
     'Rooms': 'NumberOfRooms',
     'Bedrooms' : 'NumberOfBedrooms',
     'FullBaths':'NumberOfFullBaths',
     'HalfBaths':'NumberOfHalfBaths'
    },
    axis=1,
    inplace=True
)

### Owner
# Add Owner columns; Hocking does not have Owner data, so it will be empty
hocking_subset.insert(1, "Owner", " ")
# Add LegalDescription; Hocking does not have legalDescription in the data
hocking_subset.insert(6, "LegalDescription", " ")

# Add Columns for data merge
mailing_address = hocking_subset.PropertyAddress
hocking_subset.insert(3, "MailingAddress", mailing_address)

hocking_subset['County'] = 'Hocking'

hocking_columns = [
    'ParcelNumber',
    'Owner',
    'PropertyAddress',
    'MailingAddress',
    'LandUse',    
    'Acres',
    'LegalDescription',
    'NeighborhoodCode',
    'SaleDate',
    'SalePrice',
    'YearBuilt',
    'NumberOfStories',
    'FinishedArea',
    'NumberOfRooms',
    'NumberOfBedrooms',
    'NumberOfFullBaths',
    'NumberOfHalfBaths',
    'County'
]


#hocking_columns = [
#    'ParcelNumber',
#    'Owner',
#    'PropertyAddress',
#    'MailingAddress',
#    'LandUse',    
#    'Acres',
#    'LegalDescription',
#    'NeighborhoodCode',
#    'SaleDate',
#    'SalePrice',
#    'YearBuilt',
#    'NumberOfStories',
#    'FinishedArea',
#    'NumberOfRooms',
#    'NumberOfBedrooms',
#    'NumberOfFullBaths',
#    'NumberOfHalfBaths',
#    'SchoolDistrict',
#    'Township',
#    'SaleNumber', 
#    'Basement',
#    'Frame',
#    'BasementSqft',
#    'FinishedBasement',
#    'Air',
#    'GarageSqft',
#    'GarageType',
#    'HeatType',
#    'Roof',
#    'RoofType',
#    'Grade',        
#    'LandValue',
#    'BldgValue',
#    'PropertyCard'    
#]

# Apply the column masks to the dataframe
hocking_subset = hocking_subset[hocking_columns].copy()

hocking_subset.head()
#for column in hocking_subset.columns:
#    print(column)



,Parcel Number,Number,Street,School District,Township,Corp / Village,Sale #,Month,Year,Sale Price,Sqft,Acres,Yr Built,Story,Basement,Frame,Bedrooms,BasementSqft,Fin Base,Air,GarageSqft,Type,Heat Type,Roof,RoofType,Rooms,FullBaths,HalfBaths,Grade,MHRE,NeighCode,PropClass,LandValue,BldgValue,Property Card
0,10000080900,19475.0,LINDEN,2A,4A,NaN,287.0,7,10,40000,NaN,18432.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.0,500,32840.0,NaN,http://www.realestate.co.hocking.oh.us/cards/C...
1,10000530000,NaN,NaN,2A,4A,NaN,536.0,12,10,230000,NaN,70000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7015.0,120,10970.0,NaN,http://www.realestate.co.hocking.oh.us/cards/C...
2,10000560101,20680.0,KEIFEL,2A,4A,NaN,141.0,4,10,171600,1120.0,18365.0,2002,1,B,F,2.0,1120.0,F,NaN,816.0,D,HEAT PMP,S,NaN,6.0,2.0,NaN,C,NaN,7015.0,511,16390.0,122750.0,http://www.realestate.co.hocking.oh.us/cards/C...
3,10000700000,20644.0,CUPP,2A,4A,NaN,474.0,11,10,65000,NaN,73610.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7015.0,502,24970.0,NaN,http://www.realestate.co.hocking.oh.us/cards/C...
4,10000710600,23431.0,BIG PINE,2A,4A,NaN,531.0,12,10,158950,608.0,90345.0,1993,1,NaN,L,2.0,NaN,NaN,NaN,750.0,D,FA GAS,S,GAB,NaN,1.0,NaN,C,NaN,7015.0,581,27580.0,71240.0,http://www.realestate.co.hocking.oh.us/cards/C...


,ParcelNumber,Owner,PropertyAddress,MailingAddress,LandUse,Acres,LegalDescription,NeighborhoodCode,SaleDate,SalePrice,YearBuilt,NumberOfStories,FinishedArea,NumberOfRooms,NumberOfBedrooms,NumberOfFullBaths,NumberOfHalfBaths,County
2,10000560101,,20680.0 KEIFEL,20680.0 KEIFEL,511,18365.0,,7015.0,4-01-2010,171600,2002,1,1120.0,6.0,2.0,2.0,0.0,Hocking
4,10000710600,,23431.0 BIG PINE,23431.0 BIG PINE,581,90345.0,,7015.0,12-01-2010,158950,1993,1,608.0,NaN,2.0,1.0,0.0,Hocking
5,10000900000,,20327.0 ST RT 664,20327.0 ST RT 664,511,9012.0,,7015.0,11-01-2010,60000,1970,1,1120.0,4.0,1.0,1.0,0.0,Hocking
6,10001030000,,20672.0 ST RT 664,20672.0 ST RT 664,511,4630.0,,7015.0,9-01-2010,50000,2006,1,1518.0,5.0,3.0,2.0,0.0,Hocking
9,10002600200,,23270.0 ST RT 56,23270.0 ST RT 56,513,213240.0,,7015.0,12-01-2010,120000,1953,1,926.0,5.0,1.0,1.0,0.0,Hocking


In [ ]:
### Sample County Data

# using sampleCounty in samplingsu2.py
hocking_subset = sampling.sampleCounty(hocking_subset, 0.1)

display(hocking_subset.head())

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///../data/output.sqlite')

delaware_subset.to_sql("Delaware", con=engine, if_exists='replace')
hocking_subset.to_sql("Hocking", con=engine, if_exists='replace')